In [1]:
import lightgbm
import numpy
import pandas
import random
import os
import numpy as np
def statfeature(table, key, statdict, pre=""):
   if not isinstance(key, list):
      key = [key]
   table_temp=table.groupby(key).aggregate(statdict)
   table_temp.columns = ["%s%s and %s%s" % (pre, "".join(key), col, c if isinstance(c, str) else c.__name__) for col, fun in statdict.items() for c in (fun if isinstance(fun, list) else [fun])]
   return table_temp
pandas.DataFrame.statfeature = statfeature

train_table = pandas.read_csv("final_dataset_train.tsv", sep="\t")
train_table["id"] = range(-1, -len(train_table) -1, -1)
test_table = pandas.read_csv("final_dataset_testA.tsv", sep="\t")
test_table["delta_g"] = -1

In [6]:
def align(qseq, tseq):
    qlen, tlen = len(qseq), len(tseq)
    a01 = np.zeros([qlen+1, tlen+1], dtype=np.int32)
    a10 = np.zeros([qlen+1, tlen+1], dtype=np.int32)
    a11 = np.zeros([qlen+1, tlen+1], dtype=np.int32)
    for i in range(1, qlen+1):
        for j in range(1, tlen+1):
            if qseq[i-1] == tseq[j-1]:
                a11[i, j] = max(a11[i-1, j-1] + 1, a01[i-1, j-1] + 1, a10[i-1, j-1] + 1, 0)
            else:
                a11[i, j] = max(a11[i-1, j-1] - 1, a01[i-1, j-1] - 1, a10[i-1, j-1] - 1, 0)
            a01[i, j] = max(a01[i, j-1] - 1, a11[i, j-1] - 4, 0)
            a10[i, j] = max(a10[i-1, j] - 1, a11[i-1, j] - 4, 0)
    return np.max(a11) / (qlen+0.0)

In [7]:
seq1="DIVLTQSPATLSVTPGDSVSLSCRASQSISNNLHWYQQKSHESPRLLIKYASQSISGIPSRFSGSGSGTDFTLSINSVETEDFGMYFCQQSNSWPYTFGGGTKLEIKRADAAPTVSIFPPSSEQLTSGGASVVCFLNNFYPKDINVKWKIDGSERQNGVLNSWTDQDSKDSTYSMSSTLTLTKDEYERHNSYTCEATHKTSTSPIVKSFNRNEC"
seq2="DIVLTQSPATLHKTPGDSVSLSCRASQSISNNLHWYQQKSHESPRLLIKYASQSISGIPSRFSGSGSGTDFTLSINSVETEDFGMYFCQQSNSWPYTFGGGTKLEIKRADAAPTVSIFPPSSEQLTSGGASVVCFLNNFYPKDINVKWKIDGSERQNGVLNSWTDQDSKDSTYSMSSTLTLTKDEYERHNSYTCEATHKTSTSPIVKSFNRNEC"
score=align(seq1,seq2)
print(score)

0.9813084112149533


In [13]:
train_base_feature_table = train_table.loc[:, ["id","pdb","antibody_seq_a", "antibody_seq_b", "antigen_seq"]]
for a in ["antibody_seq_a", "antibody_seq_b", "antigen_seq"]:
    #train_base_feature_table["%s length" % a] = train_base_feature_table[a].str.len()
    
    #ll=train_base_feature_table[a].str.len()
    feature=np.zeros((1706,60))
    #for b in range(1706):
    for b in range(1706):
        pdb_name=train_base_feature_table["pdb"][b]
        fasta=train_base_feature_table[a][b]
        pdb_structure_name=os.path.join("./sequence_by_chain/",pdb_name+'_sequence_chain.txt')
        f=open(pdb_structure_name,"r")
        align_score={}
        score_all=[]
        kk=0
        for line in f:
            temp=line.split()
            chain=temp[0]
            sequence=temp[1]
            score=align(fasta,sequence)
            align_score[str(kk)]=chain
            score_all.append(score)
            kk+=1
        index=score_all.index(max(score_all))
       # print(align_score[str(index)])
        fasta_name=os.path.join("./trainset_fasta/",str(b)+'_'+a+'_of_'+pdb_name+'.txt')
        f1=open(fasta_name,"w")
        print >> f1, "%s %s" %(fasta,align_score[str(index)])
        #feature[b,:]=nature_vector(fasta)
    #for c in range(60):
    #    train_base_feature_table["%s_%s" % (a, c)] = feature[:,c]
    
#train_base_feature_table = train_base_feature_table.drop(["antibody_seq_a", "antibody_seq_b", "antigen_seq"], axis=1)
test_base_feature_table = test_table.loc[:, ["id","pdb", "antibody_seq_a", "antibody_seq_b", "antigen_seq"]]
for a in ["antibody_seq_a", "antibody_seq_b", "antigen_seq"]:
    #test_base_feature_table["%s length" % a] = test_base_feature_table[a].str.len()
    #ll=test_base_feature_table[a].str.len()
    feature=np.zeros((178,60))
    for b in range(178):
        pdb_name=test_base_feature_table["pdb"][b]
        fasta=test_base_feature_table[a][b]
        pdb_structure_name=os.path.join("./sequence_by_chain/",pdb_name+'_sequence_chain.txt')
        f=open(pdb_structure_name,"r")
        align_score={}
        score_all=[]
        kk=0
        for line in f:
            temp=line.split()
            chain=temp[0]
            sequence=temp[1]
            score=align(fasta,sequence)
            align_score[str(kk)]=chain
            score_all.append(score)
            kk+=1
        index=score_all.index(max(score_all))
        fasta_name=os.path.join("./testset_fasta/",str(b)+'_'+a+'_of_'+pdb_name+'.txt')
        f1=open(fasta_name,"w")
        print >> f1, "%s %s" %(fasta,align_score[str(index)])
#        fasta=test_base_feature_table[a][b]
#        feature[b,:]=nature_vector(fasta)
   # for c in range(60):
   #     test_base_feature_table["%s_%s" % (a, c)] = feature[:,c]
#test_base_feature_table = test_base_feature_table.drop(["antibody_seq_a", "antibody_seq_b", "antigen_seq"], axis=1)

IndexError: list index out of range